### Solve IK for `Common-Rig`

In [1]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2R,r2quat,get_uv_dict_nc,get_p_target_nc
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

def normalize(angle):
	return np.arctan2(np.sin(angle),np.cos(angle))

MuJoCo version:[2.3.7]


### Parse `scene_common_rig_psyonic_aux.xml`

In [2]:
xml_path = '../asset/common_rig/scene_common_rig_psyonic_aux.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=True)
# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    body_idx = env.body_names.index(body_name)
    geom_idxs = [idx for idx,val in enumerate(env.model.geom_bodyid) if val==body_idx]
    for geom_idx in geom_idxs:
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]
print ("Done.")

dt:[0.0050] HZ:[200]
n_dof (=nv):[85]
n_geom:[75]
geom_names:['floor', 'base', 'root2spine', 'spine2neck', 'neck2rclavicle', 'neck2lclavicle', 'neck2head', 'rclavicle2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rtbase', 'rwrist2ribase', 'rwrist2rmbase', 'rwrist2rrbase', 'rwrist2rpbase', 'rtbase2rthumb', 'rthumb_l12', 'rthumb_l23', 'rthumb_l3end', 'ribase2rindex', 'rindex_l12', 'rindex_l23', 'rindex_l3end', 'rmbase2rmiddle', 'rmiddle_l12', 'rmiddle_l23', 'rmiddle_l3end', 'rmbase2rring', 'rring_l12', 'rring_l23', 'rring_l3end', 'rmbase2rpinky', 'rpinky_l12', 'rpinky_l23', 'rpinky_l3end', 'lclavicle2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2ltbase', 'lwrist2libase', 'lwrist2lmbase', 'lwrist2lrbase', 'lwrist2lpbase', 'ltbase2lthumb', 'lthumb_l12', 'lthumb_l23', 'lthumb_l3end', 'libase2lindex', 'lindex_l12', 'lindex_l23', 'lindex_l3end', 'lmbase2lmiddle', 'lmiddle_l12', 'lmiddle_l23', 'lmiddle_l3end', 'lrbase2lring', 'lring_l12', 'lring_l23', 'lring_l3end', 'lpba

### Parse bvh (Entire Sequence)

In [3]:
import numpy as np
import pickle
import cv2
import os
from bvh.skeleton import process_bvhfile, process_bvhkeyframe

data_path = "../data/bvh/VAAI_DIRECT_11_01_a_M1.bvh"
mocap = process_bvhfile(data_path, DEBUG=False)

print("Analyzing frames...")
for i in range(mocap.frames):
    new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)
print("done")

pos_header, pos = mocap.get_frames_worldpos()
rot_header, rot = mocap.get_frames_rotations()

num_frames = mocap.frames
num_joints = len(mocap.get_frame(0))
print("Total timesteps: ", num_frames)

p_full = np.zeros((num_frames, num_joints, 3))
r_full = np.zeros((num_frames, num_joints, 3))
timesteps = np.zeros(num_frames)

for i in range(num_frames):
    timesteps[i] = i * mocap.dt
    for j in range(num_joints):
        p_full[i,j,:] = np.array(pos[i][3*j+1:3*j+4]) / 100
        r_full[i,j,:] = np.array(rot[i][3*j+1:3*j+4])

p_full[:,:,1] *= -1
p_full = p_full[:,:,[1,0,2]]
r_full[:,:,1] *= -1
r_full = r_full[:,:,[1,0,2]]

Reading BVH file...
done
Building skeleton...
done
Analyzing frames...
done
Total timesteps:  215


### Set IK targets

In [4]:
qpos_data = np.zeros((num_frames, env.data.qpos.shape[0]-10))

aux_idx = list(range(0,86))
for i in (21,25,29,33,37,49,53,57,61,65):
    aux_idx.remove(i)

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='Common-Rig',viewer_width=1200,viewer_height=800,
                    viewer_hide_menus=True)
env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])

print("Total timesteps: ", num_frames)

for timestep in range(num_frames):
    p = p_full[timestep]
    r = r_full[timestep]
    uv_dict = get_uv_dict_nc(p)
    p_target = get_p_target_nc(p, uv_dict)

    env.reset()

    # Set which joints to control
    ctrl_joint_names = env.ctrl_names
    joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
    joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
    q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
    n_ctrl_joint = len(ctrl_joint_names)

    ik_body_names = ['right_pelvis','right_knee','right_ankle','left_pelvis','left_knee','left_ankle','spine','neck',
                     'right_shoulder','right_elbow','right_wrist','left_shoulder','left_elbow','left_wrist']

    ik_right_hand_names = ['rthumb_l2','rthumb_l3','rthumb_end',
                           'rindex_l1','rindex_l2','rindex_l3','rindex_end',
                           'rmiddle_l1','rmiddle_l2','rmiddle_l3','rmiddle_end',
                           'rring_l1','rring_l2','rring_l3','rring_end',
                           'rpinky_l1','rpinky_l2','rpinky_l3','rpinky_end']

    ik_left_hand_names =  ['lthumb_l2','lthumb_l3','lthumb_end',
                           'lindex_l1','lindex_l2','lindex_l3','lindex_end',
                           'lmiddle_l1','lmiddle_l2','lmiddle_l3','lmiddle_end',
                           'lring_l1','lring_l2','lring_l3','lring_end',
                           'lpinky_l1','lpinky_l2','lpinky_l3','lpinky_end']
    
    ik_target_indices = [1,2,3,6,7,8,13,15,
                        17,18,19,45,46,47,
                        40,41,42,
                        35,36,37,38,
                        30,31,32,33,
                        25,26,27,28,
                        21,22,23,24,
                        68,69,70,
                        63,64,65,66,
                        58,59,60,61,
                        53,54,55,56,
                        48,49,50,51]

    IK_Ps = [True]*len(ik_body_names)
    IK_Rs = [False]*len(ik_body_names)

    ik_p_trgts = []
    ik_p_rh_trgts = []
    ik_p_lh_trgts = []
    ik_R_trgts = []

    for body in ik_body_names:
        ik_p_trgts.append(p_target[body])

    for right_hand in ik_right_hand_names:
        ik_p_rh_trgts.append(p_target[right_hand])

    for left_hand in ik_left_hand_names:
        ik_p_lh_trgts.append(p_target[left_hand])

    for i in ik_target_indices:
        ik_R_trgts.append(rpy2R(np.radians(r[i]),[2,1,0]))

    # Reset and loop 
    q = q_ctrl_init.copy()
    body_diff = 1e3

    PERTURB_BASE_POSITION = True
    if PERTURB_BASE_POSITION:
        jntadr = env.model.body('base').jntadr[0]
        qposadr = env.model.jnt_qposadr[jntadr]
        base_pos = p[0] # modify base position
        env.data.qpos[qposadr:qposadr+3] = base_pos
        env.data.qpos[qposadr+3:qposadr+7] = r2quat(rpy2R(np.radians(r[0] - [90,0,0]),[0,1,2]))

    prev_q_pos = env.data.qpos.copy()
    FIRST_FLAG = True
    while body_diff > 1e-4:

        # Solve inverse kinematics for body
        J_list,ik_err_list = [],[]
        for ik_idx,ik_body_name in enumerate(ik_body_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            IK_P = IK_Ps[ik_idx]
            IK_R = IK_Rs[ik_idx]
            J,ik_err = env.get_ik_ingredients(
                body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=IK_P,IK_R=IK_R)
            J_list.append(J)
            ik_err_list.append(ik_err)
        J_stack      = np.vstack(J_list)
        ik_err_stack = np.hstack(ik_err_list)
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        q_rev = q[0:37].copy()
        np.clip(q_rev, env.ctrl_joint_mins[0:37], env.ctrl_joint_maxs[0:37], out=q_rev)
        q[0:37] = q_rev.copy()
        
        # FK
        env.data.qpos[joint_idxs_fwd] = q
        q_env = env.data.qpos.copy()
        body_diff = np.linalg.norm(q_env - prev_q_pos)

        env.forward(q=q_env,INCREASE_TICK=True)
        prev_q_pos=q_env.copy()
        
        if env.loop_every(tick_every=10):
            # Plot world frame
            env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                    PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
            
            # Text information
            env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3),PLOT_AXIS=False,
                    label='Tick:[%d]'%(env.tick))
            
            # Plot bodies
            env.plot_body_T(body_name='base',PLOT_AXIS=False,axis_len=0.5,axis_width=0.01)
            for body_name in env.body_names:
                p,R = env.get_pR_body(body_name=body_name)
                env.plot_T(p=p,R=R,PLOT_AXIS=False,axis_len=0.05,axis_width=0.005)
                
            # Plot IK target
            for ik_idx,ik_body_name in enumerate(ik_body_names):
                ik_p_trgt = ik_p_trgts[ik_idx]
                ik_R_trgt = ik_R_trgts[ik_idx]
                env.plot_T(p=ik_p_trgt,R=ik_R_trgt,
                        PLOT_AXIS=True,axis_len=0.1,axis_width=0.002)

    right_wrist_pos = env.get_p_body(body_name='right_wrist')
    left_wrist_pos = env.get_p_body(body_name='left_wrist')
    hand_diff = 1e3
    env.tick = 0

    while hand_diff > 1e-4 and env.tick < 2000:

        # Solve inverse kinematics for body
        J_list,ik_err_list = [],[]
        for ik_idx,ik_body_name in enumerate(ik_body_names):
            ik_p_trgt = ik_p_trgts[ik_idx]
            ik_R_trgt = ik_R_trgts[ik_idx]
            IK_P = IK_Ps[ik_idx]
            IK_R = IK_Rs[ik_idx]
            J,ik_err = env.get_ik_ingredients(
                body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=IK_P,IK_R=IK_R)
            J_list.append(J)
            ik_err_list.append(ik_err)
        J_stack      = np.vstack(J_list)
        ik_err_stack = np.hstack(ik_err_list)
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]

        # Solve inverse kinematics for right hand
        J_rh_list,ik_err_rh_list = [],[]
        for ik_rh_idx,ik_right_hand_name in enumerate(ik_right_hand_names):
            ik_p_trgt = ik_p_rh_trgts[ik_rh_idx] + right_wrist_pos
            ik_R_trgt = ik_R_trgts[ik_rh_idx+len(ik_body_names)]
            IK_P = True
            IK_R = False
            J,ik_err = env.get_ik_ingredients(
                body_name=ik_right_hand_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=IK_P,IK_R=IK_R)
            J_rh_list.append(J)
            ik_err_rh_list.append(ik_err)
        J_stack      = np.vstack(J_rh_list)
        ik_err_stack = np.hstack(ik_err_rh_list)
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        q_rev = q[37:77].copy()
        np.clip(q_rev, env.ctrl_joint_mins[37:77], env.ctrl_joint_maxs[37:77], out=q_rev)
        q[37:77] = q_rev.copy()

        # Solve inverse kinematics for left hand
        J_lh_list,ik_err_lh_list = [],[]
        for ik_lh_idx,ik_left_hand_name in enumerate(ik_left_hand_names):
            ik_p_trgt = ik_p_lh_trgts[ik_lh_idx] + left_wrist_pos
            ik_R_trgt = ik_R_trgts[ik_lh_idx+len(ik_body_names)+len(ik_right_hand_names)]
            IK_P = True
            IK_R = False
            J,ik_err = env.get_ik_ingredients(
                body_name=ik_left_hand_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                IK_P=IK_P,IK_R=IK_R)
            J_lh_list.append(J)
            ik_err_lh_list.append(ik_err)
        J_stack      = np.vstack(J_lh_list)
        ik_err_stack = np.hstack(ik_err_lh_list)
        dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
        q = q + dq[joint_idxs_jac]
        q_rev = q[37:77].copy()
        np.clip(q_rev, env.ctrl_joint_mins[37:77], env.ctrl_joint_maxs[37:77], out=q_rev)
        q[37:77] = q_rev.copy()

        # FK
        env.data.qpos[joint_idxs_fwd] = q
        curr_q_pos = env.data.qpos.copy()
        hand_diff = np.linalg.norm(curr_q_pos - prev_q_pos)

        env.forward(q=curr_q_pos,INCREASE_TICK=True)
        prev_q_pos = curr_q_pos.copy()
            
    # Close MuJoCo viewer
    env.close_viewer()

    qpos_refined = normalize(np.array(curr_q_pos[aux_idx]))
    qpos_data[timestep,:] = qpos_refined

    if timestep % 10 == 0:
        print("Timestep: ", timestep)

p_root = p_full[:,0,:]

pkl_data = {
    'length': num_frames,
    'root': p_root,
    'qpos': qpos_data
}
with open('../data/pkl/' + data_path.split('/')[-1].split('.')[0] + '.pkl', 'wb') as f:
    pickle.dump(pkl_data, f)

print("Rigging Completed")

Total timesteps:  215
Timestep:  0
Timestep:  10
Timestep:  20
Timestep:  30
Timestep:  40
Timestep:  50
Timestep:  60
Timestep:  70
Timestep:  80
Timestep:  90
Timestep:  100
Timestep:  110
Timestep:  120
Timestep:  130
Timestep:  140
Timestep:  150
Timestep:  160
Timestep:  170
Timestep:  180
Timestep:  190
Timestep:  200
Timestep:  210
Rigging Completed


### Repeat for all 80 sequences

In [ ]:
import numpy as np
import pickle
import cv2
import os
from bvh.skeleton import process_bvhfile, process_bvhkeyframe

aux_idx = list(range(0,86))
for i in (21,25,29,33,37,49,53,57,61,65):
    aux_idx.remove(i)

data_types = ["a","b","c"]

for data_type in data_types:
    if data_type == "a":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "b":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","16_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]

    elif data_type == "c":
        data_numbers = ["01_01","02_01","03_01","04_01","05_01","06_02","07_01","08_01","09_02","10_01",
                        "11_01","12_02","13_01","14_01","15_02","17_01","18_01","19_01","20_01",
                        "21_01","22_01","23_01","24_01","25_02","26_01","27_01"]
          
    for data_number in data_numbers:
        data_path = "../data/bvh/VAAI_DIRECT_%s_%s_M1.bvh" % (data_number, data_type)
        mocap = process_bvhfile(data_path, DEBUG=False)

        for i in range(mocap.frames):
            new_frame = process_bvhkeyframe(mocap.keyframes[i], mocap.root, mocap.dt * i)

        pos_header, pos = mocap.get_frames_worldpos()
        rot_header, rot = mocap.get_frames_rotations()

        num_frames = mocap.frames
        num_joints = len(mocap.get_frame(0))

        p_full = np.zeros((num_frames, num_joints, 3))
        r_full = np.zeros((num_frames, num_joints, 3))
        timesteps = np.zeros(num_frames)

        for i in range(num_frames):
            timesteps[i] = i * mocap.dt
            for j in range(num_joints):
                p_full[i,j,:] = np.array(pos[i][3*j+1:3*j+4]) / 100
                r_full[i,j,:] = np.array(rot[i][3*j+1:3*j+4])

        p_full[:,:,1] *= -1
        p_full = p_full[:,:,[1,0,2]]
        r_full[:,:,1] *= -1
        r_full = r_full[:,:,[1,0,2]]

        qpos_data = np.zeros((num_frames, env.data.qpos.shape[0]-10))

        # Initialize MuJoCo viewer
        env.init_viewer(viewer_title='Common-Rig',viewer_width=1200,viewer_height=800,
                            viewer_hide_menus=True)
        env.update_viewer(azimuth=152,distance=3.0,elevation=-30,lookat=[0.02,-0.03,0.8])

        print("Total timesteps: ", num_frames)

        for timestep in range(num_frames):
            p = p_full[timestep]
            r = r_full[timestep]
            uv_dict = get_uv_dict_nc(p)
            p_target = get_p_target_nc(p, uv_dict)

            env.reset()

            # Set which joints to control
            ctrl_joint_names = env.ctrl_names
            joint_idxs_fwd = env.get_idxs_fwd(joint_names=ctrl_joint_names)
            joint_idxs_jac = env.get_idxs_jac(joint_names=ctrl_joint_names)
            q_ctrl_init = env.get_qpos_joints(ctrl_joint_names)
            n_ctrl_joint = len(ctrl_joint_names)

            ik_body_names = ['right_pelvis','right_knee','right_ankle','left_pelvis','left_knee','left_ankle','spine','neck',
                            'right_shoulder','right_elbow','right_wrist','left_shoulder','left_elbow','left_wrist']

            ik_right_hand_names = ['rthumb_l2','rthumb_l3','rthumb_end',
                                'rindex_l1','rindex_l2','rindex_l3','rindex_end',
                                'rmiddle_l1','rmiddle_l2','rmiddle_l3','rmiddle_end',
                                'rring_l1','rring_l2','rring_l3','rring_end',
                                'rpinky_l1','rpinky_l2','rpinky_l3','rpinky_end']

            ik_left_hand_names =  ['lthumb_l2','lthumb_l3','lthumb_end',
                                'lindex_l1','lindex_l2','lindex_l3','lindex_end',
                                'lmiddle_l1','lmiddle_l2','lmiddle_l3','lmiddle_end',
                                'lring_l1','lring_l2','lring_l3','lring_end',
                                'lpinky_l1','lpinky_l2','lpinky_l3','lpinky_end']
            
            ik_target_indices = [1,2,3,6,7,8,13,15,
                                17,18,19,45,46,47,
                                40,41,42,
                                35,36,37,38,
                                30,31,32,33,
                                25,26,27,28,
                                21,22,23,24,
                                68,69,70,
                                63,64,65,66,
                                58,59,60,61,
                                53,54,55,56,
                                48,49,50,51]

            IK_Ps = [True]*len(ik_body_names)
            IK_Rs = [False]*len(ik_body_names)

            ik_p_trgts = []
            ik_p_rh_trgts = []
            ik_p_lh_trgts = []
            ik_R_trgts = []

            for body in ik_body_names:
                ik_p_trgts.append(p_target[body])

            for right_hand in ik_right_hand_names:
                ik_p_rh_trgts.append(p_target[right_hand])

            for left_hand in ik_left_hand_names:
                ik_p_lh_trgts.append(p_target[left_hand])

            for i in ik_target_indices:
                ik_R_trgts.append(rpy2R(np.radians(r[i]),[2,1,0]))

            # Reset and loop 
            q = q_ctrl_init.copy()
            body_diff = 1e3

            PERTURB_BASE_POSITION = True
            if PERTURB_BASE_POSITION:
                jntadr = env.model.body('base').jntadr[0]
                qposadr = env.model.jnt_qposadr[jntadr]
                base_pos = p[0] # modify base position
                env.data.qpos[qposadr:qposadr+3] = base_pos
                env.data.qpos[qposadr+3:qposadr+7] = r2quat(rpy2R(np.radians(r[0] - [90,0,0]),[0,1,2]))

            prev_q_pos = env.data.qpos.copy()
            FIRST_FLAG = True
            while body_diff > 1e-4:

                # Solve inverse kinematics for body
                J_list,ik_err_list = [],[]
                for ik_idx,ik_body_name in enumerate(ik_body_names):
                    ik_p_trgt = ik_p_trgts[ik_idx]
                    ik_R_trgt = ik_R_trgts[ik_idx]
                    IK_P = IK_Ps[ik_idx]
                    IK_R = IK_Rs[ik_idx]
                    J,ik_err = env.get_ik_ingredients(
                        body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                        IK_P=IK_P,IK_R=IK_R)
                    J_list.append(J)
                    ik_err_list.append(ik_err)
                J_stack      = np.vstack(J_list)
                ik_err_stack = np.hstack(ik_err_list)
                dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                q = q + dq[joint_idxs_jac]
                q_rev = q[0:37].copy()
                np.clip(q_rev, env.ctrl_joint_mins[0:37], env.ctrl_joint_maxs[0:37], out=q_rev)
                q[0:37] = q_rev.copy()
                
                # FK
                env.data.qpos[joint_idxs_fwd] = q
                q_env = env.data.qpos.copy()
                body_diff = np.linalg.norm(q_env - prev_q_pos)

                env.forward(q=q_env,INCREASE_TICK=True)
                prev_q_pos=q_env.copy()
                
                if env.loop_every(tick_every=10):
                    # Plot world frame
                    env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                            PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
                    
                    # Text information
                    env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3),PLOT_AXIS=False,
                            label='Tick:[%d]'%(env.tick))
                    
                    # Plot bodies
                    env.plot_body_T(body_name='base',PLOT_AXIS=False,axis_len=0.5,axis_width=0.01)
                    for body_name in env.body_names:
                        p,R = env.get_pR_body(body_name=body_name)
                        env.plot_T(p=p,R=R,PLOT_AXIS=False,axis_len=0.05,axis_width=0.005)
                        
                    # Plot IK target
                    for ik_idx,ik_body_name in enumerate(ik_body_names):
                        ik_p_trgt = ik_p_trgts[ik_idx]
                        ik_R_trgt = ik_R_trgts[ik_idx]
                        env.plot_T(p=ik_p_trgt,R=ik_R_trgt,
                                PLOT_AXIS=True,axis_len=0.1,axis_width=0.002)

            right_wrist_pos = env.get_p_body(body_name='right_wrist')
            left_wrist_pos = env.get_p_body(body_name='left_wrist')
            hand_diff = 1e3
            env.tick = 0

            while hand_diff > 1e-4 and env.tick < 2000:

                # Solve inverse kinematics for body
                J_list,ik_err_list = [],[]
                for ik_idx,ik_body_name in enumerate(ik_body_names):
                    ik_p_trgt = ik_p_trgts[ik_idx]
                    ik_R_trgt = ik_R_trgts[ik_idx]
                    IK_P = IK_Ps[ik_idx]
                    IK_R = IK_Rs[ik_idx]
                    J,ik_err = env.get_ik_ingredients(
                        body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                        IK_P=IK_P,IK_R=IK_R)
                    J_list.append(J)
                    ik_err_list.append(ik_err)
                J_stack      = np.vstack(J_list)
                ik_err_stack = np.hstack(ik_err_list)
                dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                q = q + dq[joint_idxs_jac]

                # Solve inverse kinematics for right hand
                J_rh_list,ik_err_rh_list = [],[]
                for ik_rh_idx,ik_right_hand_name in enumerate(ik_right_hand_names):
                    ik_p_trgt = ik_p_rh_trgts[ik_rh_idx] + right_wrist_pos
                    ik_R_trgt = ik_R_trgts[ik_rh_idx+len(ik_body_names)]
                    IK_P = True
                    IK_R = False
                    J,ik_err = env.get_ik_ingredients(
                        body_name=ik_right_hand_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                        IK_P=IK_P,IK_R=IK_R)
                    J_rh_list.append(J)
                    ik_err_rh_list.append(ik_err)
                J_stack      = np.vstack(J_rh_list)
                ik_err_stack = np.hstack(ik_err_rh_list)
                dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                q = q + dq[joint_idxs_jac]
                q_rev = q[37:77].copy()
                np.clip(q_rev, env.ctrl_joint_mins[37:77], env.ctrl_joint_maxs[37:77], out=q_rev)
                q[37:77] = q_rev.copy()

                # Solve inverse kinematics for left hand
                J_lh_list,ik_err_lh_list = [],[]
                for ik_lh_idx,ik_left_hand_name in enumerate(ik_left_hand_names):
                    ik_p_trgt = ik_p_lh_trgts[ik_lh_idx] + left_wrist_pos
                    ik_R_trgt = ik_R_trgts[ik_lh_idx+len(ik_body_names)+len(ik_right_hand_names)]
                    IK_P = True
                    IK_R = False
                    J,ik_err = env.get_ik_ingredients(
                        body_name=ik_left_hand_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                        IK_P=IK_P,IK_R=IK_R)
                    J_lh_list.append(J)
                    ik_err_lh_list.append(ik_err)
                J_stack      = np.vstack(J_lh_list)
                ik_err_stack = np.hstack(ik_err_lh_list)
                dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
                q = q + dq[joint_idxs_jac]
                q_rev = q[37:77].copy()
                np.clip(q_rev, env.ctrl_joint_mins[37:77], env.ctrl_joint_maxs[37:77], out=q_rev)
                q[37:77] = q_rev.copy()

                # FK
                env.data.qpos[joint_idxs_fwd] = q
                curr_q_pos = env.data.qpos.copy()
                hand_diff = np.linalg.norm(curr_q_pos - prev_q_pos)

                env.forward(q=curr_q_pos,INCREASE_TICK=True)
                prev_q_pos = curr_q_pos.copy()
                    
            # Close MuJoCo viewer
            env.close_viewer()

            qpos_refined = normalize(np.array(curr_q_pos[aux_idx]))
            qpos_data[timestep,:] = qpos_refined

            if timestep % 10 == 0:
                print("Timestep: ", timestep)

        p_root = p_full[:,0,:]

        pkl_data = {
            'length': num_frames,
            'root': p_root,
            'qpos': qpos_data
        }
        with open('../data/pkl/' + data_path.split('/')[-1].split('.')[0] + '.pkl', 'wb') as f:
            pickle.dump(pkl_data, f)

        print("%s%s Rigging Completed" % (data_number[:2], data_type))